#### Libraries

In [2]:
import openai
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import sys
import re
import time

import os
#import openai
#openai.Model.list()

#### Convert chemical data into better csv

In [ ]:
biocreative_disease_corpus = ET.parse('Biocreative_dataset/chemical/TestSet.tmChem.BioC.xml')

# find the root
root = biocreative_disease_corpus.getroot()

data_frame_articles = pd.DataFrame()
# find each Title and abstract
#all_tags = root.findall('document')
for doc in root.findall('document'):
    # Get the pubmed id, title and abstract text
    pubmed_id = doc.find('id').text
    title_passage = doc.findall('passage')[0]
    title_text = title_passage.find('text').text
    abstract_passage = doc.findall('passage')[1]
    abstract_text = abstract_passage.find('text').text
    # we create a list of data
    array_of_data = [pubmed_id,title_text,abstract_text]
    # append to dataframe
    data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)

print(data_frame_articles)

data_frame_articles.to_csv('Biocreative_dataset/chemical/TestSet.tmChem.BioC.csv',index=False)

#### Since we have CHEBIS in our datasets we need to find them and ask chatGPT again

In [31]:
biocreative_disease_corpus = ET.parse('Biocreative_dataset/chemical/TestSet.tmChem.BioC.xml')

# find the root
root = biocreative_disease_corpus.getroot()

data_frame_articles_CHEBI = pd.DataFrame()
# find each Title and abstract
#all_tags = root.findall('document')
for doc in root.findall('document'):
    pubmed_id = doc.find('id').text
    title_passage = doc.findall('passage')[0]
    title_text = title_passage.find('text').text
    abstract_passage = doc.findall('passage')[1]
    abstract_text = abstract_passage.find('text').text
    title_annotations = title_passage.findall('annotation')
    abstract_annotations = abstract_passage.findall('annotation')
    found_in_titles = False
    for annotation in title_annotations:
        if annotation.find('infon[@key="CHEBI"]') is not None:
            array_of_data = [pubmed_id,title_text,abstract_text]
            data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
            found_in_titles = True
            break
    if not found_in_titles:
        for annotation in abstract_annotations:
            if annotation.find('infon[@key="CHEBI"]') is not None:
                array_of_data = [pubmed_id,title_text,abstract_text]
                data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
                break


data_frame_articles.to_csv('Biocreative_dataset/chemical/TestSet.tmChem.BioC.CHEBI.csv',index=False)

C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2576014179.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2576014179.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2576014179.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=T

In [32]:
data_frame_articles = pd.read_csv('Biocreative_dataset/chemical/TestSet.tmChem.BioC.CHEBI.csv')
#text = "Disease|MESH ID stress ulcers|D013286delirium|D003693famotidine-associated delirium|C566034"
start = "20959502"
go = True

data_frame_new = pd.read_csv("results_chatGPT/chemical/TestSet.tmChem.BioC.chatGPTResults.CHEBI.csv")
count_limit = 0
for index,row in data_frame_articles.iterrows():
    title_and_abstract = row.title+row.abstract
    #new_row = [row.pubmed_id,reply.content]
    print(row.pubmed_id)
    if go:
        try:
            chat = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a NER tool"},
                    #{"role": "assistant", "content": "Extract only disease entities and find their correct MESH ids,store them in csv format delimiter is | in \"Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost. Although all of the currently available H2-receptor antagonists have shown the propensity to cause delirium, only two previously reported cases have been associated with famotidine. The authors report on six cases of famotidine-associated delirium in hospitalized patients who cleared completely upon removal of famotidine. The pharmacokinetics of famotidine are reviewed, with no change in its metabolism in the elderly population seen. The implications of using famotidine in elderly persons are discussed.\""},
                    {"role": "assistant", "content": "Extract only Chemicals and find their correct MeSH ids and CHEBI ids,display the result only in this format name_of_chemical|MeSH_id|CHEBI_id without headers and nothing else.Text is: \""+title_and_abstract+"\""},
                ]
            )
            reply = chat.choices[0].message
            new_row = [row.pubmed_id,reply.content]
            data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)
            count_limit = count_limit + 1
        except:
            print("EXCEPTION HAPPENED")
            data_frame_new.to_csv('results_chatGPT/chemical/TestSet.tmChem.BioC.chatGPTResults.CHEBI.csv',index=False)
            time.sleep(60)
            count_limit = 0
            chat = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a NER tool"},
                    #{"role": "assistant", "content": "Extract only disease entities and find their correct MESH ids,store them in csv format delimiter is | in \"Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost. Although all of the currently available H2-receptor antagonists have shown the propensity to cause delirium, only two previously reported cases have been associated with famotidine. The authors report on six cases of famotidine-associated delirium in hospitalized patients who cleared completely upon removal of famotidine. The pharmacokinetics of famotidine are reviewed, with no change in its metabolism in the elderly population seen. The implications of using famotidine in elderly persons are discussed.\""},
                    {"role": "assistant", "content": "Extract only Chemicals and find their correct MeSH ids and CHEBI ids,display the result only in this format name_of_chemical|MeSH_id|CHEBI_id without headers and nothing else.Text is: \""+title_and_abstract+"\""},
                ]
            )
            reply = chat.choices[0].message
            new_row = [row.pubmed_id,reply.content]
            data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)
            count_limit = count_limit + 1
    if count_limit == 3:
        data_frame_new.to_csv('results_chatGPT/chemical/TestSet.tmChem.BioC.chatGPTResults.CHEBI.csv',index=False)
        time.sleep(60)
        count_limit = 0

    if str(row.pubmed_id) == str(start):
        go = True

data_frame_new.to_csv('results_chatGPT/chemical/TestSet.tmChem.BioC.chatGPTResults.CHEBI.csv',index=False)
#data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)



15605432


C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2477354459.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)


15609701


C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2477354459.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)


16323982


...

C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2477354459.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)


19108278


C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2477354459.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)


20046642


C:\Users\matev\AppData\Local\Temp\ipykernel_19656\2477354459.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_frame_new = data_frame_new.append(pd.Series(new_row,index=["pubmed_id","result"]),ignore_index=True)


### Data preprocessing

In [3]:
# --- Results chatGPT --- 
article_results_chatGPT = pd.read_csv('./results_chatGPT-3_5/chemical/TestSet.tmChem.BioC.chatGPTResults.csv')
#print(article_results_chatGPT)

# Since we need to do everything again we need to ignore the values that contain chebi
articles_containing_chebi = pd.read_csv('./results_chatGPT-3_5/chemical/TestSet.tmChem.BioC.chatGPTResults.CHEBI.csv')
article_identifiers = articles_containing_chebi.iloc[:, 0].astype(str).tolist()

print(article_identifiers)
# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in article_results_chatGPT.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row[1]
    if str(row[0]) not in article_identifiers: 
        for chatGPTanno in all_annotations.split('\n'):
            if len(chatGPTanno) > 0:
                    if len(chatGPTanno.split('|')) > 2:
                        chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1],chatGPTanno.strip().split('|')[2]))
                    else:     
                        chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1],None))
                    chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
        dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
        dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2


# We get the biocreative dataset to compare with our findings
biocreative_chemical_corpus = ET.parse('Biocreative_dataset/chemical/TestSet.tmChem.BioC.xml')

# find the root
root = biocreative_chemical_corpus.getroot()
all_correct_entities = 0
data_frame_articles = pd.DataFrame()
# find all results and convert them into a better format
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for doc in root.findall('document'):
    # Get the pubmed id, title and abstract text
    pubmed_id = doc.find('id').text
    title_passage = doc.findall('passage')[0]
    title_annotations = title_passage.findall('annotation')
    annotations_set = set()
    annotations_set_2 = set()
    if str(pubmed_id) not in article_identifiers:
        for annotation in title_annotations:
            # We noticed that in the chemical dataset they also use CHEBI links
            if annotation.find('infon[@key="MESH"]') is not None:
                annotations_set.add((annotation.find("text").text,annotation.find('infon[@key="MESH"]').text.replace('MESH:','')))
            elif annotation.find('infon[@key="CHEBI"]') is not None:
                annotations_set.add((annotation.find("text").text,"CHEBI:"+annotation.find('infon[@key="CHEBI"]').text))
            else:
                annotations_set.add((annotation.find("text").text,"None"))
            annotations_set_2.add(annotation.find("text").text)
        #title_text = title_passage.find('text').text
        abstract_passage = doc.findall('passage')[1]
        abstract_annotations = abstract_passage.findall('annotation')
        for anno in abstract_annotations:
            # We noticed that in the chemical dataset they also use CHEBI links
            if anno.find('infon[@key="MESH"]') is not None:
                annotations_set.add((anno.find("text").text,anno.find('infon[@key="MESH"]').text.replace('MESH:','')))
            elif anno.find('infon[@key="CHEBI"]') is not None:
                annotations_set.add((anno.find("text").text,"CHEBI:"+anno.find('infon[@key="CHEBI"]').text))
            else:
                annotations_set.add((anno.find("text").text,"None"))
            annotations_set_2.add(anno.find("text").text)
        #abstract_text = abstract_passage.find('text').text
        # we create a list of data
        #array_of_data = [pubmed_id,title_text,abstract_text]
        # append to dataframe
        #data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
        dictionary_of_results[str(pubmed_id)] = annotations_set
        dictionary_of_results_without_MESH[str(pubmed_id)] = annotations_set_2
        all_correct_entities += len(annotations_set_2)
print(len(dictionary_of_results))
print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))

path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/chemical/results_without_chebi/chatGPT_chemical_statistics_without_MESH.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/chemical/results_without_chebi/chatGPT_chemical_statistics_with_MESH.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/chemical/results_without_chebi/confusion_matrix_chemical_pure_entities.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/chemical/results_without_chebi/confusion_matrix_chemical_entities_test_contains.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/chemical/results_without_chebi/confusion_matrix_chemical_entities_test_contains_and_chatgpt_contains.csv"

['439781', '23872883', '24067251', '24132704', '24333387', '24434397', '24464946', '24659727', '24742750', '24840785', '25054547', '25084821', '2782734', '35781', '689020', '931801', '1117341', '1158089', '1361574', '1423336', '1615846', '1760851', '1858969', '2239937', '2383364', '2907577', '2974281', '3685052', '3708328', '4082192', '4631913', '6806735', '7315949', '8160791', '8188982', '8829135', '8996419', '9272404', '9759693', '9869655', '12372954', '12498738', '12677626', '15265979', '15282950', '15605432', '15609701', '16323982', '16723784', '17028363', '17356399', '17379047', '17484470', '18165598', '18340638', '18422462', '19108278', '20046642']
442
442
442
442


### DATA PREPROCESSING CHATGPT-3.5 ONLY CHEBI

In [ ]:
# --- Results chatGPT --- 
article_results_chatGPT = pd.read_csv('./results_chatGPT-3_5/chemical/TestSet.tmChem.BioC.chatGPTResults.csv')
#print(article_results_chatGPT)

# Since we need to do everything again we need to ignore the values that contain chebi
articles_containing_chebi = pd.read_csv('./results_chatGPT-3_5/chemical/TestSet.tmChem.BioC.chatGPTResults.CHEBI.csv')
article_identifiers = articles_containing_chebi.iloc[:, 0].astype(str).tolist()

print(article_identifiers)
# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in article_results_chatGPT.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row[1]
    if str(row[0]) in article_identifiers: 
        for chatGPTanno in all_annotations.split('\n'):
            if len(chatGPTanno) > 0:
                    if len(chatGPTanno.split('|')) > 2:
                        chatGPT_annotations.add((chatGPTanno.split('|')[0],chatGPTanno.split('|')[1],chatGPTanno.split('|')[2]))
                    else:     
                        chatGPT_annotations.add((chatGPTanno.split('|')[0],chatGPTanno.split('|')[1],None))
                    chatGPT_annotations_2.add(chatGPTanno.split('|')[0])
        dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
        dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2


# We get the biocreative dataset to compare with our findings
biocreative_chemical_corpus = ET.parse('Biocreative_dataset/chemical/TestSet.tmChem.BioC.xml')

# find the root
root = biocreative_chemical_corpus.getroot()
all_correct_entities = 0
data_frame_articles = pd.DataFrame()
# find all results and convert them into a better format
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for doc in root.findall('document'):
    # Get the pubmed id, title and abstract text
    pubmed_id = doc.find('id').text
    title_passage = doc.findall('passage')[0]
    title_annotations = title_passage.findall('annotation')
    annotations_set = set()
    annotations_set_2 = set()
    if str(pubmed_id) in article_identifiers:
        for annotation in title_annotations:
            # We noticed that in the chemical dataset they also use CHEBI links
            if annotation.find('infon[@key="MESH"]') is not None:
                annotations_set.add((annotation.find("text").text,annotation.find('infon[@key="MESH"]').text.replace('MESH:','')))
            elif annotation.find('infon[@key="CHEBI"]') is not None:
                annotations_set.add((annotation.find("text").text,"CHEBI:"+annotation.find('infon[@key="CHEBI"]').text))
            else:
                annotations_set.add((annotation.find("text").text,"None"))
            annotations_set_2.add(annotation.find("text").text)
        #title_text = title_passage.find('text').text
        abstract_passage = doc.findall('passage')[1]
        abstract_annotations = abstract_passage.findall('annotation')
        for anno in abstract_annotations:
            # We noticed that in the chemical dataset they also use CHEBI links
            if anno.find('infon[@key="MESH"]') is not None:
                annotations_set.add((anno.find("text").text,anno.find('infon[@key="MESH"]').text.replace('MESH:','')))
            elif anno.find('infon[@key="CHEBI"]') is not None:
                annotations_set.add((anno.find("text").text,"CHEBI:"+anno.find('infon[@key="CHEBI"]').text))
            else:
                annotations_set.add((anno.find("text").text,"None"))
            annotations_set_2.add(anno.find("text").text)
        #abstract_text = abstract_passage.find('text').text
        # we create a list of data
        #array_of_data = [pubmed_id,title_text,abstract_text]
        # append to dataframe
        #data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
        dictionary_of_results[str(pubmed_id)] = annotations_set
        dictionary_of_results_without_MESH[str(pubmed_id)] = annotations_set_2
        all_correct_entities += len(annotations_set_2)
print(len(dictionary_of_results))
print(len(dictionary_of_chatGPT_results))

path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/chemical/results_with_chebi/chatGPT_chemical_statistics_without_MESH.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/chemical/results_with_chebi/chatGPT_chemical_statistics_with_MESH.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/chemical/results_with_chebi/confusion_matrix_chemical_pure_entities.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/chemical/results_with_chebi/confusion_matrix_chemical_entities_test_contains.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/results_with_chebi/chemical/confusion_matrix_chemical_entities_test_contains_and_chatgpt_contains.csv"

### DATA PREPROCESSING CHATGPT4

In [4]:
# --- Results chatGPT --- 
article_results_chatGPT = pd.read_csv('./results_chatgpt-4/chemical/TestSet.tmChem.BioC.chatGPT4Results.csv')
#print(article_results_chatGPT)
# Since we need to do everything again we need to ignore the values that contain chebi
articles_containing_chebi = pd.read_csv('./results_chatgpt-4/chemical/TestSet.tmChem.BioC.chatGPT4Results_CHEBI.csv')
article_identifiers = articles_containing_chebi.iloc[:, 0].astype(str).tolist()


# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in article_results_chatGPT.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row[1]
    if str(row[0]) not in article_identifiers:
        for chatGPTanno in str(all_annotations).split('\n'):
            print(chatGPTanno)
            if len(chatGPTanno) > 0 and chatGPTanno != 'nan':
                    if len(chatGPTanno.split('|')) > 2:
                        chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1],chatGPTanno.strip().split('|')[2]))
                    else:     
                        chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1],None))
                    chatGPT_annotations_2.add(chatGPTanno.split('|')[0])
        dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
        dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2


# We get the biocreative dataset to compare with our findings
biocreative_chemical_corpus = ET.parse('Biocreative_dataset/chemical/TestSet.tmChem.BioC.xml')

# find the root
root = biocreative_chemical_corpus.getroot()
all_correct_entities = 0
data_frame_articles = pd.DataFrame()
# find all results and convert them into a better format
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for doc in root.findall('document'):
    # Get the pubmed id, title and abstract text
    pubmed_id = doc.find('id').text
    title_passage = doc.findall('passage')[0]
    title_annotations = title_passage.findall('annotation')
    annotations_set = set()
    annotations_set_2 = set()
    print(pubmed_id)
    if str(pubmed_id) not in article_identifiers:
        for annotation in title_annotations:
            # We noticed that in the chemical dataset they also use CHEBI links
            if annotation.find('infon[@key="MESH"]') is not None:
                annotations_set.add((annotation.find("text").text,annotation.find('infon[@key="MESH"]').text.replace('MESH:','')))
            elif annotation.find('infon[@key="CHEBI"]') is not None:
                annotations_set.add((annotation.find("text").text,"CHEBI:"+annotation.find('infon[@key="CHEBI"]').text))
            else:
                annotations_set.add((annotation.find("text").text,"None"))
            annotations_set_2.add(annotation.find("text").text)
        #title_text = title_passage.find('text').text
        abstract_passage = doc.findall('passage')[1]
        abstract_annotations = abstract_passage.findall('annotation')
        for anno in abstract_annotations:
            # We noticed that in the chemical dataset they also use CHEBI links
            if anno.find('infon[@key="MESH"]') is not None:
                annotations_set.add((anno.find("text").text,anno.find('infon[@key="MESH"]').text.replace('MESH:','')))
            elif anno.find('infon[@key="CHEBI"]') is not None:
                annotations_set.add((anno.find("text").text,"CHEBI:"+anno.find('infon[@key="CHEBI"]').text))
            else:
                annotations_set.add((anno.find("text").text,"None"))
            annotations_set_2.add(anno.find("text").text)
        #abstract_text = abstract_passage.find('text').text
        # we create a list of data
        #array_of_data = [pubmed_id,title_text,abstract_text]
        # append to dataframe
        #data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
        dictionary_of_results[str(pubmed_id)] = annotations_set
        dictionary_of_results_without_MESH[str(pubmed_id)] = annotations_set_2
        all_correct_entities += len(annotations_set_2)
print(len(dictionary_of_results))
print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results_without_MESH))
print(len(dictionary_of_chatGPT_results_without_MESH))

path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-4/chemical/results_without_chebi/chatGPT_chemical_statistics_without_MESH.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-4/chemical/results_without_chebi/chatGPT_chemical_statistics_with_MESH.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-4/chemical/confusion_matrix_chemical_pure_entities.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/chemical/confusion_matrix_chemical_entities_test_contains.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/chemical/confusion_matrix_chemical_entities_test_contains_and_chatgpt_contains.csv"

Famotidine|D005242
Histamine H2-receptor|D006632
tacrolimus|D019818
prednisolone|D011239
cyclosporine|D016572
corticosteroids|D003240
methamphetamine|D008694
Parkinson's disease|D010300
levodopa|D007980
basal ganglia|D001480
P2X3 receptor|Q1493754
NK1 receptor|Q411436
cyclophosphamide|D003520
Suramin|D013524
GR 82334|C067285
saline|D012343
urothelium|D008184
suburothelium|No exact MESH ID found
clopidogrel|D019197
hepatotoxicity|D006487
acute hepatitis|D006525
Bortezomib|D03150
Dexamethasone|D003907
Bisphenol A|D015242
Acetylcholinesterase|D000137
lenalidomide|D000068877
dexamethasone|D003907
thalidomide|D013792
bortezomib|D000068720
ifosfamide|D007363
mesna|D008741
sodium 2-sulfanylethanesulfonate|D008741
alkylating nitrogen mustard|D009188
antineoplasmic agent|D000995
hemorrhagic cystitis|D006470
sister chromatid exchanges|D012878
proliferation rate index|No MESH found
Mitotic Index|D008958
genotoxicity|D005943
cytostatic action|D016864
levodopa|D007980
everolimus|D000068877
tacrolim

# Analysis of the data (Only entities)
_______

### PURE CORRECT ENTITIES

In [18]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper():
                pure_correct_entity += 1
                counter += 1
                break
                
    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 1228
All correct entities: 1638
Percentage(%): 74.96
Number of excess entities (incorrect): 651


### If the test entity is contained in the CHATGPT entity

In [19]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.strip().upper():
                correct_entity_with_test_contain += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 1357
All correct entities: 1638
Percentage(%): 82.84
Number of excess entities (incorrect): 522


### If test entities can be found in CHATGPTS response and vice versa

In [20]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.strip().upper() or actual.strip().upper() in test.upper():
                correct_entity_with_test_contain_and_vice_versa += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 1389
All correct entities: 1638
Percentage(%): 84.79
Number of excess entities (incorrect): 490


# Analysis of data (with MESH)
____

## Pure correct entities

In [21]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual[0].strip().upper() == test[0].upper() and (actual[1] == test[1] or actual[2] == test[1]):
                pure_correct_entity += 1
                counter += 1
                break
                
    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 410
All correct entities: 1638
Percentage(%): 25.03
Number of excess entities (incorrect): 1469


### If the test entity is contained in the CHATGPT entity

In [22]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if (actual[0].strip().upper() == test[0].upper() or test[0].upper() in actual[0].strip().upper()) and (actual[1] == test[1] or actual[2] == test[1]):
                correct_entity_with_test_contain += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 423
All correct entities: 1638
Percentage(%): 25.82
Number of excess entities (incorrect): 1456


### If test entities can be found in CHATGPTS response and vice versa

In [23]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if (actual[0].strip().upper() == test[0].upper() or test[0].upper() in actual[0].strip().upper() or actual[0].strip().upper() in test[0].upper()) and (actual[1] == test[1] or actual[2] == test[1]):
                correct_entity_with_test_contain_and_vice_versa += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 427
All correct entities: 1638
Percentage(%): 26.06
Number of excess entities (incorrect): 1452


# Analysis confusion matrix
_____

## Preprocessing

In [24]:
### Create correct dictionaries
pairs_test_set = set()
pairs_chatGPT_set = []
unique_chatGPT_ids = set()
unique_test_set = set()

# get all meshes from CHATGPT and their entities
for key in dictionary_of_chatGPT_results:
    for mesh_id in dictionary_of_chatGPT_results[key]:
        pairs_chatGPT_set.append((mesh_id[0],mesh_id[1],mesh_id[2]))
        if mesh_id[2] is not None:
            unique_chatGPT_ids.add(mesh_id[2])
        unique_chatGPT_ids.add(mesh_id[1])

# get all unique meshes from test set
for key in dictionary_of_results:
    for mesh_id in dictionary_of_results[key]:
        pairs_test_set.add((mesh_id[0],mesh_id[1]))
        unique_test_set.add(mesh_id[1])

list_of_test_meshes = list(pairs_test_set)
unique_test_set = list(unique_test_set)
list_of_chatGPT_MESHES = list(unique_chatGPT_ids)

### Function to create dataframe

In [25]:
def createRowForDataframe(key,array,all_chatGPT_unique_ids):
    array_of_values = {}
    array_of_values["test_MESH_ID"] = key 
    for element in all_chatGPT_unique_ids:
        counter = 0
        for ground_base_id in array:
            if element == ground_base_id:
                counter += 1
        array_of_values[element] = counter
    return array_of_values



## Pure correct entities

In [26]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if 'CHEBI' in test_entity[1] and test_entity[0] == entity_set[0]:
            array_of_mesh_ids.append(entity_set[2])
        elif test_entity[0] == entity_set[0]:
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_MESH_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_17536\693908864.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17536\693908864.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17536\693908864.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_o

#### Clean confusion matrix and save it into a csv

In [27]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_MESH_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_pure,index=False)

## If the test entity is contained in the CHATGPT entity

In [28]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if "CHEBI" in test_entity[1] and (test_entity[0] == entity_set[0] or test_entity[0] in entity_set[0]):
            array_of_mesh_ids.append(entity_set[2])
        elif test_entity[0] == entity_set[0] or test_entity[0] in entity_set[0]:
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_MESH_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_17536\3606498467.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17536\3606498467.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17536\3606498467.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matri

#### Clean confusion matrix and save it into a csv

In [29]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_MESH_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_lapover,index=False)

### If test entities can be found in CHATGPTS response and vice versa

In [30]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if "CHEBI" in test_entity[1] and (test_entity[0] == entity_set[0] or test_entity[0] in entity_set[0] or entity_set[0] in test_entity[0]):
            array_of_mesh_ids.append(entity_set[2])
        if test_entity[0] == entity_set[0] or test_entity[0] in entity_set[0] or entity_set[0] in test_entity[0]:
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_MESH_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_17536\2422684792.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17536\2422684792.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17536\2422684792.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matri

#### Clean confusion matrix and save it into a csv

In [31]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_MESH_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_full_lapover,index=False)